In [37]:
import ufl
import matplotlib.pyplot as plt
import time
import numpy as np
from dolfin import *
comm = MPI.comm_world  # MPI communicator

hs_ratio = 0.0  # water level in crevasse (normalized with crevasse height)
# water level at right terminus (normalized with glacier thickness)
hw_ratio = 0.5
prec_ratio = 0.08  # depth of pre-crack (normalized with glacier thickness)
energy_thsd = 62  # 0.6348

lx, ly, lz = 750, 500, 125
L, B, H = lx, ly, lz

# ---------------------------------------------------------------------------------
# MATERIAL PARAMETERS     ------------------------------------------------
# ---------------------------------------------------------------------------------

rho_ice = 917  # density of ice (kg/m^3)
rho_H2O = 1000  # density of freshwater (kg/m^3)
rho_sea = 1020  # density of seawater (kg/m^3)
grav = 9.81  # gravity acceleration (m/s**2)
E = 9.5e9  # Young's modulus (Pa)
nu = 0.35  # Poisson's ratio
mu = E / 2 / (1 + nu)  # shear modulus
K = E / 3 / (1 - 2 * nu)  # bulk modulus
lmbda = K - 2 / 3 * mu  # Lame's first parameter
KIc = 0.1e6  # critical stress intensity factor (Pa*m^0.5)
lc = .610  # nonlocal length scale (m)
# lc = 0.1  # nonlocal length scale (m)
eta = 5e1  # viscous regularization parameter (N*s/m)
g1c = (KIc**2) * (1 - nu**2) / E  # critical strain energy release rate (Pa*m)
g2c = (
    10 * g1c
)  # assumed critical strain energy release rate for mode II fracture (Pa*m)
hw = hw_ratio * H  # water level at terminus (absolute height)
ci = 1
sigmac = 0.1185e6  # critical stress only for stress based method

# ---------------------------------------------------------------------------------
# SIMULATION PARAMETERS        ------------------------------------------------
# ---------------------------------------------------------------------------------
time_elapsed = 0
time_total = 350  # arbitrary
delta_t = 1.0  # time increment
time_counter = 0
output_increment = 5

Dcr = 0.6
air, water, ice = 0, 1, 2
water_left, water_right = 3, 4
right = 2

# ---------------------------------------------------------------------------------
# PARAMETERS FOR PARALLEL COMPUTATIONS   -----------------------------
# ---------------------------------------------------------------------------------

rank = comm.Get_rank()  # number of current process
size = comm.Get_size()  # total number of processes


def mwrite(filename, my_list):
    MPI.barrier(comm)
    if rank == 0:
        with open(filename, "w") as f:
            for item in my_list:
                f.write("%s" % item)


def mprint(*argv):
    if rank == 0:
        out = ""
        for arg in argv:
            out = out + str(arg)
        # this forces program to output when run in parallel
        print(out, flush=True)


# ---------------------------------------------------------------------------------
# SET SOME COMMON FENICS FLAGS     ---------------------------------------
# ---------------------------------------------------------------------------------
set_log_level(50)
parameters["form_compiler"]["optimize"] = True
parameters["form_compiler"]["cpp_optimize"] = True

In [38]:

# ---------------------------------------------------------------------------------
# MATERIAL MODEL  --------------------------------------------------------
# ---------------------------------------------------------------------------------
def epsilon(u):
    return 0.5 * (grad(u) + grad(u).T)


def sigma(u):
    return 2.0 * mu * epsilon(u) + lmbda * tr(epsilon(u)) * Identity(len(u))

# -----------------------------------------------------------------------------
# 3D eigen decomposition
# -----------------------------------------------------------------------------


def invariants_principal(A):
    """Principal invariants of (real-valued) tensor A.
    https://doi.org/10.1007/978-3-7091-0174-2_3
    """
    i1 = ufl.tr(A)
    i2 = (ufl.tr(A)**2 - ufl.tr(A * A)) / 2
    i3 = ufl.det(A)
    return i1, i2, i3


def invariants_main(A):
    """Main invariants of (real-valued) tensor A.
    https://doi.org/10.1007/978-3-7091-0174-2_3
    """
    j1 = ufl.tr(A)
    j2 = ufl.tr(A * A)
    j3 = ufl.tr(A * A * A)
    return j1, j2, j3


def get_eigenstate(A):
    """Eigenvalues and eigenprojectors of the 3x3 (real-valued) tensor A.
    Provides the spectral decomposition A = sum_{a=0}^{2} λ_a * E_a
    with eigenvalues λ_a and their associated eigenprojectors E_a = n_a^R x n_a^L
    ordered by magnitude.
    The eigenprojectors of eigenvalues with multiplicity n are returned as 1/n-fold projector.
    Note: Tensor A must not have complex eigenvalues!
    """
    if ufl.shape(A) != (3, 3):
        raise RuntimeError(
            f"Tensor A of shape {ufl.shape(A)} != (3, 3) is not supported!")
    #
    eps = 1.0e-10
    #
    A = ufl.variable(A)
    #
    # --- determine eigenvalues λ0, λ1, λ2
    #
    # additively decompose: A = tr(A) / 3 * I + dev(A) = q * I + B
    q = ufl.tr(A) / 3
    B = A - q * ufl.Identity(3)
    # observe: det(λI - A) = 0  with shift  λ = q + ω --> det(ωI - B) = 0 = ω**3 - j * ω - b
    # == -I2(B) for trace-free B, j < 0 indicates A has complex eigenvalues
    j = ufl.tr(B * B) / 2
    b = ufl.tr(B * B * B) / 3  # == I3(B) for trace-free B
    p = 2 / ufl.sqrt(3) * ufl.sqrt(j + eps ** 2)  # eps: MMM
    r = 4 * b / p ** 3
    r = ufl.Max(ufl.Min(r, +1 - eps), -1 + eps)  # eps: LMM, MMH
    phi = ufl.acos(r) / 3
    # sorted eigenvalues: λ0 <= λ1 <= λ2
    λ0 = q + p * ufl.cos(phi + 2 / 3 * ufl.pi)  # low
    λ1 = q + p * ufl.cos(phi + 4 / 3 * ufl.pi)  # middle
    λ2 = q + p * ufl.cos(phi)  # high
    return as_tensor([[λ2, 0, 0], [0, λ1, 0], [0, 0, λ0]])

# ---------------------------------------------------------------------------------
# ENERGY CALCULATIONS        ------------------------------------------------
# ---------------------------------------------------------------------------------

# Apply some scalar-to-scalar mapping `f` to each component of `T`:


def applyElementwise(f, T):
    from ufl import shape

    sh = shape(T)
    if len(sh) == 0:
        return f(T)
    fT = []
    for i in range(0, sh[0]):
        fT += [applyElementwise(f, T[i, i])]
    return as_tensor([[fT[0], 0, 0], [0, fT[1], 0], [0, 0, fT[2]]])


def split_plus_minus(T):
    x_plus = applyElementwise(lambda x: 0.5 * (abs(x) + x), T)
    x_minus = applyElementwise(lambda x: 0.5 * (abs(x) - x), T)
    return x_plus, x_minus


def safeSqrt(x):
    return sqrt(x + DOLFIN_EPS)


def get_energy(unew,D):
    stress_plus, stress_minus = split_plus_minus(get_eigenstate(sigma(unew)))
    energy_expr = ci * (
        (stress_plus[0, 0] / sigmac) ** 2 + (stress_plus[1, 1] /
                                             sigmac) ** 2 + (stress_plus[2, 2] / sigmac) ** 2 - 1
    )
    energy_expr = ufl.Max(energy_expr, 0)
    # Apply the threshold condition
    energy_expr = ufl.conditional(
        ufl.le(energy_expr, energy_thsd), 0, energy_expr)

    energy = project(energy_expr, D, solver_type="cg",
                     preconditioner_type="hypre_euclid").vector()[:]
    # energy = assemble(energy_expr*TestFunction(D)*dx())
    return energy

In [39]:
#  ▄▄       ▄▄  ▄▄▄▄▄▄▄▄▄▄▄  ▄▄▄▄▄▄▄▄▄▄▄  ▄▄        ▄
# ▐░░▌     ▐░░▌▐░░░░░░░░░░░▌▐░░░░░░░░░░░▌▐░░▌      ▐░▌
# ▐░▌░▌   ▐░▐░▌▐░█▀▀▀▀▀▀▀█░▌ ▀▀▀▀█░█▀▀▀▀ ▐░▌░▌     ▐░▌
# ▐░▌▐░▌ ▐░▌▐░▌▐░▌       ▐░▌     ▐░▌     ▐░▌▐░▌    ▐░▌
# ▐░▌ ▐░▐░▌ ▐░▌▐░█▄▄▄▄▄▄▄█░▌     ▐░▌     ▐░▌ ▐░▌   ▐░▌
# ▐░▌  ▐░▌  ▐░▌▐░░░░░░░░░░░▌     ▐░▌     ▐░▌  ▐░▌  ▐░▌
# ▐░▌   ▀   ▐░▌▐░█▀▀▀▀▀▀▀█░▌     ▐░▌     ▐░▌   ▐░▌ ▐░▌
# ▐░▌       ▐░▌▐░▌       ▐░▌     ▐░▌     ▐░▌    ▐░▌▐░▌
# ▐░▌       ▐░▌▐░▌       ▐░▌ ▄▄▄▄█░█▄▄▄▄ ▐░▌     ▐░▐░▌
# ▐░▌       ▐░▌▐░▌       ▐░▌▐░░░░░░░░░░░▌▐░▌      ▐░░▌
#  ▀         ▀  ▀         ▀  ▀▀▀▀▀▀▀▀▀▀▀  ▀        ▀▀

In [40]:
def pff_problem(mesh, pold_adaptive, time_counter):
    # ---------------------------------------------------------------------------------
    # FUNCTION SPACES  ---------------------------------------------------------
    # ---------------------------------------------------------------------------------
    V = VectorFunctionSpace(mesh, "CG", 1)  # displacement shape function
    D = FunctionSpace(mesh, "DG", 0)  # phase-field shape function
    # ---------------------------------------------------------------------------------
    # BOUNDARIES AND MEASURES    ------------------------------------------------
    # ---------------------------------------------------------------------------------
    # lx,ly,lz=750,500,125
    front = CompiledSubDomain("near(x[0], 0)")
    back = CompiledSubDomain("near(x[0], L)", L=L)
    left = CompiledSubDomain("near(x[1], 0)")
    right_csd = CompiledSubDomain("near(x[1], B)", B=B)
    bottom = CompiledSubDomain("near(x[2], 0)")
    top = CompiledSubDomain("near(x[2], H)", H=H)

    bottom_roller = DirichletBC(V.sub(2), Constant(0), bottom)
    left_roller = DirichletBC(V.sub(1), Constant(0), left)
    front_roller = DirichletBC(V.sub(0), Constant(0), front)
    back_roller = DirichletBC(V.sub(0), Constant(0), back)

    disp_bc = [bottom_roller, left_roller, front_roller, back_roller]

    boundaries = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
    right_csd.mark(boundaries, right)
    ds = Measure("ds", subdomain_data=boundaries)
    # ---------------------------------------------------------------------------------
    # FUNCTIONS    --------------------------------------------------------------
    # ---------------------------------------------------------------------------------

    u, v = TrialFunction(V), TestFunction(V)
    p, q = TrialFunction(D), TestFunction(D)

    unew, uold = Function(V, name="displacement"), Function(V)
    pnew, pold = Function(D, name="damage"), Function(D)
    cdf = Function(D, name='energy')

    # ---------------------------------------------------------------------------------
    # UPDATE FUNCTIONS    --------------------------------------------------------
    # ---------------------------------------------------------------------------------

    if time_counter > 0:
        pold.assign(pold_adaptive)

    # ---------------------------------------------------------------------------------
    # DISPLACEMENT PROBLEM      -----------------------------------------------
    # ---------------------------------------------------------------------------------
    disp_a = inner(((1 - pold) ** 2 + 1e-4) * sigma(u), epsilon(v)) * dx
    disp_L = inner(v, Constant((0, 0, -rho_ice * grav))) * dx

    b_hw = Expression(
        (0, "(h - x[1] > 0 ?-rho_H2O * grav*(h - x[1]) : 0)", 0),
        h=hw,
        rho_H2O=rho_H2O,
        grav=grav,
        degree=1,
    )

    if hw_ratio > 0:
        disp_L += inner(v, b_hw) * ds(right)  # terminus pressure
    unew = Function(V, name="displacement")
    disp_problem = LinearVariationalProblem(disp_a, disp_L, unew, disp_bc)
    disp_solver = LinearVariationalSolver(disp_problem)

    prm_disp = disp_solver.parameters
    prm_disp["linear_solver"] = "cg"
    prm_disp["preconditioner"] = "hypre_euclid"
    # ---------------------------------------------------------------------------------
    # PHASE FIELD PROBLEM      -----------------------------------------------
    # ---------------------------------------------------------------------------------
    phase_a = (eta / delta_t + 1 / lc + 2 * cdf) * inner(p, q) * dx + lc * inner(
        grad(p), grad(q)
    ) * dx
    phase_L = inner(q, 2 * cdf) * dx + eta / delta_t * inner(q, pold) * dx
    phase_problem = LinearVariationalProblem(phase_a, phase_L, pnew)
    phase_solver = LinearVariationalSolver(phase_problem)

    prm_phase = phase_solver.parameters
    prm_phase["linear_solver"] = "cg"
    prm_phase["preconditioner"] = "hypre_euclid"
    # ---------------------------------------------------------------------------------
    # START ITERATIONS    -----------------------------------------------------
    # ---------------------------------------------------------------------------------
    disp_solver.solve()
    # update history variable --------------------------------------------------
    cdf.vector().vec().array[:] = get_energy(unew,D)[:]
    # ToDo: kappa = np.fmax(energy, kappa)
    phase_solver.solve()
    # Clip the damage solution to 0-1 ------------------------------------------
    pnew.vector().vec().array[:] = np.clip(pnew.vector()[:], 0, 1)
    # pnew will be max(pold,pnew)-----------------------------------------------
    # pnew.vector().vec().array[:] = np.maximum(pnew.vector()[:], pold.vector()[:])
    # update counter -----------------------------------------------------------
    xdmf.write(unew, time_counter)
    xdmf.write(pnew, time_counter)
    xdmf.write(cdf, time_counter)
    
    return pnew,cdf

In [41]:
# mesh = BoxMesh(Point(0,0,0), Point(L,B,H),10,10,10)
mesh = Mesh()
with XDMFFile(comm, "mesh/tetra.xdmf") as infile:
    infile.read(mesh)

xdmf = XDMFFile(comm, "output/solution.xdmf")
xdmf.parameters["functions_share_mesh"] = False
xdmf.parameters["rewrite_function_mesh"] = True
xdmf.parameters["flush_output"] = True

In [42]:
cpp_code = """
#include <pybind11/pybind11.h>
#include <dolfin/adaptivity/adapt.h>
#include <dolfin/function/Function.h>
#include <dolfin/mesh/Mesh.h>
namespace py = pybind11;
PYBIND11_MODULE(SIGNATURE, m)
{
m.def("adapt", [](const dolfin::Function &function,
          std::shared_ptr<const dolfin::Mesh> adapted_mesh,
                  bool interpolate){
             return dolfin::adapt(function, adapted_mesh, interpolate);});
}
"""
m = compile_cpp_code(cpp_code)


def adaptFunction(f, mesh, interp=True):
    return m.adapt(f, mesh, interp)

In [43]:
def transfer(_p, mesh):
    _p = Function(adaptFunction(_p._cpp_object, mesh))
    return _p

In [44]:
def estimate(cdf_old,cdf_new):
    error_estimate = cdf_new.vector()[:] - cdf_old.vector()[:]
    return error_estimate

def get_markers(marker_array, mesh, phi, Y0, cdf, alpa, target_hmin):
    min_achieved = False

    marker = MeshFunction("bool", mesh, mesh.topology().dim())
    marker.set_all(False)
    # ---------------------------------------------------------------------------------
    # Scheme S1    -----------------------------------------------------
    # ---------------------------------------------------------------------------------
    #  elements are refined, for which the value of error indicator is greater than α·max(error indicator)
    maximum_value = MPI.max(comm,max(marker_array))
    alpha_max = alpa * maximum_value
    marker.array()[marker_array > alpha_max] = True
    # ---------------------------------------------------------------------------------
    # Scheme S2    -----------------------------------------------------
    # ---------------------------------------------------------------------------------
    marker.array()[phi.vector()[:] > 0.1] = True
    # ---------------------------------------------------------------------------------
    # Scheme S3    -----------------------------------------------------
    # ---------------------------------------------------------------------------------
    marker.array()[cdf.vector()[:] < Y0] = False
    # ---------------------------------------------------------------------------------
    # Scheme S4    -----------------------------------------------------
    # ---------------------------------------------------------------------------------
    cell_dia = 2 * Circumradius(mesh)
    DG = FunctionSpace(mesh, "DG", 0)
    dia_vector = project(cell_dia, DG, solver_type="cg",
        preconditioner_type="hypre_euclid").vector()[:]
    marker.array()[dia_vector < target_hmin] = False
    # ---------------------------------------------------------------------------------
    # ---------------------------------------------------------------------------------
    DG = None
    del DG

    min_achieved =    np.all(np.invert(marker.array()))
    return marker, min_achieved

In [45]:
time_counter = 0
start = time.time()
p_adaptive = Function(FunctionSpace(mesh, "DG", 0))
cdf_old = Function(FunctionSpace(mesh, "DG", 0), name="energy")
cdf_new = Function(FunctionSpace(mesh, "DG", 0), name="energy")
while time_elapsed <= time_total:
# while time_elapsed <= 10:
    # ---------------------------------------------------------------------------------
    # SOLVE      -----------------------------------------------
    # ---------------------------------------------------------------------------------
    p_adaptive, cdf_new = pff_problem(mesh, p_adaptive, time_counter)
    # ---------------------------------------------------------------------------------
    # ESTIMATE      -----------------------------------------------
    # ---------------------------------------------------------------------------------
    error_estimate = estimate(cdf_old,cdf_new)
    # ---------------------------------------------------------------------------------
    # MARK     -----------------------------------------------
    # ---------------------------------------------------------------------------------
    marker, min_achieved = get_markers(error_estimate, mesh, p_adaptive, energy_thsd, 
                                       cdf_new, alpa=0.9, target_hmin=12)
    # ---------------------------------------------------------------------------------
    # REFINE      -----------------------------------------------
    # ---------------------------------------------------------------------------------
    mesh = refine(mesh, marker)
    # ---------------------------------------------------------------------------------
    # TRANSFER      -----------------------------------------------
    # ---------------------------------------------------------------------------------
    p_adaptive = transfer(p_adaptive, mesh)
    cdf_old.assign(transfer(cdf_new, mesh))
    # ---------------------------------------------------------------------------------
    time_counter += 1
    time_elapsed += delta_t
    mprint(
        "step: {0:3}, vertices: {1:9.0f}, cells: {2:9.0f}, hmin: {3:5.2f} time: {4:6.0f}".format(
            time_counter,
            FunctionSpace(mesh, "CG", 1).dim(),
            FunctionSpace(mesh, "DG", 0).dim(),
            MPI.min(comm,mesh.hmin()),
            time.time() - start,
        )
    )

Calling FFC just-in-time (JIT) compiler, this may take some time.
step:   1, vertices:      6037, cells:     31450, hmin:  6.64 time:      8
step:   2, vertices:      6422, cells:     33444, hmin:  3.40 time:     13
step:   3, vertices:      6888, cells:     35991, hmin:  1.80 time:     18
step:   4, vertices:      7092, cells:     37016, hmin:  1.60 time:     21
step:   5, vertices:      7363, cells:     38493, hmin:  1.60 time:     23
step:   6, vertices:      7706, cells:     40355, hmin:  1.60 time:     26
step:   7, vertices:      8085, cells:     42393, hmin:  1.52 time:     29
step:   8, vertices:      8576, cells:     45030, hmin:  1.52 time:     32
step:   9, vertices:      8999, cells:     47364, hmin:  0.94 time:     35
step:  10, vertices:      9638, cells:     50785, hmin:  0.94 time:     39
step:  11, vertices:      9948, cells:     52443, hmin:  0.94 time:     43
step:  12, vertices:     10451, cells:     55214, hmin:  0.94 time:     47
step:  13, vertices:     11197, ce